## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/women-clothing-accessories.csv', sep='\t', usecols=[0, 1])

In [ ]:
data.sample(5)

,review,sentiment
65812,"Все пришло, на этом спасибо",positive
56981,"Заказывала чёрный,пришёл синий,ткань тонкая ,в...",neautral
54457,Воняет просто отвратительно !!!! Просто ужасно...,neautral
54133,"Платье коротковато на рост 159, один рукав уже...",neautral
30056,Пойдёт для сельской местности,neautral


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90000 entries, 0 to 89999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     90000 non-null  object
 1   sentiment  90000 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB


In [ ]:
data['sentiment'].value_counts()

neautral    30000
positive    30000
negative    30000
Name: sentiment, dtype: int64

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf (диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
report_list=[]
def generate_df_data(vectorizer, params, precision, recall, fscore, accuracy, punct) :
    report_list.append({'Vectorizer' : vectorizer, 
                        'Vectorizer_parameters' : params, 
                        'Use punctuation' : punct,
                        'Precision' : precision, 
                        'Recall' : recall,
                        'F1-score' : fscore,
                        'Accuracy' : accuracy})
    

In [ ]:
import nltk 

nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 8.2 MB 29.8 MB/s 


In [ ]:
import pymorphy2
from pymorphy2 import MorphAnalyzer

import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
"""
приведение к нижнему регистру
удаление пунктуации optional
лемматизация
удаление стоп-слов будет производиться далее
"""

morph = pymorphy2.MorphAnalyzer()

def make_preprocessed(row, isPunct) :
    #make lower & remove punctuations (optional) & make lemmatization
    tokenized = word_tokenize(row.lower())
    return " ".join([morph.parse(token)[0].normal_form 
                     for token in tokenized 
                     if isPunct or token[0] not in string.punctuation])

def make_preprocessed_with_punkt(row) :
    return make_preprocessed(row, True)

def make_preprocessed_without_punkt(row) :
    return make_preprocessed(row, False)

In [ ]:
data['review_preproc_without_punct'] = data['review'].apply(make_preprocessed_without_punkt)
data['review_preproc_with_punct'] = data['review'].apply(make_preprocessed_with_punkt)

In [ ]:
data.sample(5)

,review,sentiment,review_preproc_without_punct,review_preproc_with_punct
87849,"Хорошая блузка, на 42 взяли L, смотрится хорош...",positive,хороший блузка на 42 взять l смотреться хорошо...,"хороший блузка , на 42 взять l , смотреться хо..."
73883,"на 105кг 117-107-133см xxxl - всамый раз, не т...",positive,на 105кг 117-107-133см xxxl всамый раз не тесн...,"на 105кг 117-107-133см xxxl - всамый раз , не ..."
58383,"в груди и правда маловато, в то время когда ру...",neautral,в грудь и правда маловатый в то время когда ру...,"в грудь и правда маловатый , в то время когда ..."
7168,"Джинсы заказываю второй раз, качество не очень...",negative,джинсы заказывать второй раз качество не очень...,"джинсы заказывать второй раз , качество не оче..."
60354,Платье просто божественное и по качеству и вне...,positive,платье просто божественный и по качество и вне...,платье просто божественный и по качество и вне...


In [ ]:
import itertools

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support as prfs_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.review, data.sentiment, test_size=0.33, random_state=42)
X_train_proc, X_test_proc, y_train_proc, y_test_proc = train_test_split(data.review_preproc_without_punct, data.sentiment, test_size=0.33, random_state=42)
X_train_proc_punct, X_test_proc_punct, y_train_proc_punct, y_test_proc_punct = train_test_split(data.review_preproc_with_punct, data.sentiment, test_size=0.33, random_state=42)

Вынесем в функции общие куски кода для применения векторайзеров

In [ ]:
noise = stopwords.words('russian')


def generate_ngram_full_range(start, end) :
    return itertools.combinations_with_replacement(range(start, end), 2)


def generate_ngram_range(start, end) :
    return [(start, e) for e in range(start, end)]


def naive_bayes_classifier(vectorized_x_train, y_train, vectorized_x_test, y_test) :
    clf = MultinomialNB()
    clf.fit(vectorized_x_train, y_train)

    pred = clf.predict(vectorized_x_test)
    cur_report = classification_report(y_test, pred)
    
    return pred, cur_report
    

def get_models(get_vectorizer, gen_ngram_range, X_train, y_train, X_test, y_test, ngram_start = 1, ngram_end = 8, punct = False, params=(), print_cur_res = False) :
    bestCharacteristics, best = None, 0.0

    for ngram_range in gen_ngram_range(ngram_start, ngram_end):
        vectorizer_params = params + (ngram_range)
        pred, report, vectorizer_df_params = get_vectorizer(X_train, y_train, X_test, y_test, vectorizer_params, print_cur_res)

        precision, recall, fscore, support = prfs_score(y_test, pred, average='macro')
        accuracy = accuracy_score(y_test, pred)
        cur = np.mean(np.array([precision, recall, fscore, accuracy]))
        if cur > best :
            best = cur
            bestCharacteristics = (ngram_range, report, 
                                   vectorizer_df_params[0], vectorizer_df_params[1], precision, recall, fscore, accuracy)

    generate_df_data(bestCharacteristics[2], 
                     bestCharacteristics[3], 
                     bestCharacteristics[4], 
                     bestCharacteristics[5], 
                     bestCharacteristics[6], 
                     bestCharacteristics[7], 
                     punct)
    print('----------------------\nBEST:\nngram_range', bestCharacteristics[0], '\n', bestCharacteristics[1])

**n-граммы**

`def ngrams(X_train, y_train, X_test, y_test, analyzer = 'word', punct = False, stop_words = None) :`

1.   `analyzer = 'word'` (стоит как анализатор по умолчанию) мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2.   `analyzer = 'char'` символьные n-граммы (диапазон для n подбирайте самостоятельно)





**CountVectorizer с разными параметрами**

Векторизатор преобразует слово или набор слов в числовой вектор, понятный алгоритму машинного обучения, который привык работать с числовыми табличными данными.

Для обучения используем обучающую выборку `x_train`, но в отличие от классификатора мы используем метод `fit_transform()`: сначала обучаем наш векторизатор, а потом сразу применяем его к нашему набору данных.

- **Стоп-слова** - это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе. Для модели это просто шум. А шум нужно убирать. По аналогичной причине убирают и пунктуацию.
(проверим, как отличаются модели, построенные без стоп-слов и пунктуации и с ними)



```
class sklearn.feature_extraction.text.CountVectorizer(*, 
                                                      input='content', 
                                                      encoding='utf-8', 
                                                      decode_error='strict', 
                                                      strip_accents=None, 
                                                      lowercase=True, 
                                                      preprocessor=None, 
                                                      tokenizer=None, 
                                                      stop_words=None, 
                                                      token_pattern='(?u)\b\w\w+\b', 
                                                      ngram_range=(1, 1), 
                                                      analyzer='word', 
                                                      max_df=1.0, 
                                                      min_df=1, 
                                                      max_features=None, 
                                                      vocabulary=None, 
                                                      binary=False, 
                                                      dtype=<class 'numpy.int64'>)
```



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def get_count_vectorizer(X_train, y_train, X_test, y_test, vectorizer_params=(), print_res = False) :
    analyzer, stop_words, ngram_range_start, ngram_range_end = vectorizer_params
    ngram_range = (ngram_range_start, ngram_range_end)
    smart_vectorizer = CountVectorizer(analyzer = analyzer, ngram_range = ngram_range, stop_words = stop_words) 
    smart_vectorized_x_train = smart_vectorizer.fit_transform(X_train)
    smart_vectorized_x_test = smart_vectorizer.transform(X_test)

    pred, report = naive_bayes_classifier(smart_vectorized_x_train, y_train, smart_vectorized_x_test, y_test)
    if print_res :
        print('ngram size: ', ngram_range[1] - ngram_range[0] + 1, '\nngram_range: ', ngram_range, '\nvectorized x_train shape: ', smart_vectorized_x_train.shape)
        print(report)

    vect_pararams = smart_vectorizer.get_params()
    params = ('CountVectorizer', 
              ['analyzer : ' + vect_pararams['analyzer'], 'ngram_range : ' + str(vect_pararams['ngram_range'])])
    
    return pred, report, params

**CountVectorizer word analizer**

Так как результат не зависит от порядка слов в текстах, то говорят, что такая модель представления текстов в виде векторов получается из гипотезы представления текста как мешка слов

Модель "мешок слов" фактически делает нам вектор не для слова, а для документа. При этом для векторного представления слов в такой модели используется стандартный подход: one-hot encoding. Пусть число различных слов равно  K . Сопоставим слову с номером  i  вектор длины  K , в котором  i -тая координата равна единице, а все остальные — нулям. Недостатком one-hot encoding является то, что по векторным представлениям нельзя судить о схожести смысла слов. Также вектора имеют очень большой размер, из-за чего их неэффективно хранить в памяти.

- word analizer  &  delete stop_words  &  without punctuation

In [ ]:
analizer, stop_words, punct = 'word', noise, False
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc, y_train, X_test_proc, y_test, 1, 7, False, ('word', noise), True)

ngram size:  1 
ngram_range:  (1, 1) 
vectorized x_train shape:  (60300, 25695)
              precision    recall  f1-score   support

    neautral       0.58      0.65      0.61     10026
    negative       0.71      0.62      0.66      9819
    positive       0.81      0.83      0.82      9855

    accuracy                           0.70     29700
   macro avg       0.70      0.70      0.70     29700
weighted avg       0.70      0.70      0.70     29700

ngram size:  2 
ngram_range:  (1, 2) 
vectorized x_train shape:  (60300, 330831)
              precision    recall  f1-score   support

    neautral       0.61      0.63      0.62     10026
    negative       0.71      0.66      0.68      9819
    positive       0.82      0.86      0.84      9855

    accuracy                           0.71     29700
   macro avg       0.71      0.71      0.71     29700
weighted avg       0.71      0.71      0.71     29700

ngram size:  3 
ngram_range:  (1, 3) 
vectorized x_train shape:  (60300, 9024

- word analizer  &  delete stop_words  &  with punctuation

In [ ]:
analizer, stop_words, punct = 'word', noise, True
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc_punct, y_train, X_test_proc_punct, y_test, 1, 7, True, ('word', noise))

----------------------
BEST:
ngram_range (1, 2) 
               precision    recall  f1-score   support

    neautral       0.61      0.63      0.62     10026
    negative       0.71      0.66      0.68      9819
    positive       0.82      0.86      0.84      9855

    accuracy                           0.71     29700
   macro avg       0.71      0.71      0.71     29700
weighted avg       0.71      0.71      0.71     29700



- word analizer  &  not delete stop_words  &  without punctuation

In [ ]:
analizer, stop_words, punct = 'word', None, False
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc, y_train, X_test_proc, y_test, 1, 7, False, ('word', None))

----------------------
BEST:
ngram_range (1, 6) 
               precision    recall  f1-score   support

    neautral       0.63      0.64      0.63     10026
    negative       0.71      0.71      0.71      9819
    positive       0.87      0.85      0.86      9855

    accuracy                           0.73     29700
   macro avg       0.74      0.73      0.73     29700
weighted avg       0.73      0.73      0.73     29700



- word analizer  &  not delete stop_words  &  with punctuation

In [ ]:
analizer, stop_words, punct = 'word', None, True
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc_punct, y_train, X_test_proc_punct, y_test, 1, 7, True, ('word', None))

----------------------
BEST:
ngram_range (1, 3) 
               precision    recall  f1-score   support

    neautral       0.62      0.66      0.64     10026
    negative       0.71      0.69      0.70      9819
    positive       0.87      0.86      0.86      9855

    accuracy                           0.73     29700
   macro avg       0.74      0.73      0.73     29700
weighted avg       0.74      0.73      0.73     29700



- На данных, полностью предобработанных по списку, при увеличении диапазона размеров n-грамм качество модели меняется незначительно и не увеличивается с увеличением размерности. 
Видно, что без n-грамм с относительно маленьким n, качество модели ухудшается. 
Даже при большом размере промежутка взятых n-грамм, при увеличении стартового н, модель работает хуже

- Наличие пунктуации не влияет на точность модели в случае мешка n-грамм

- Присутствие в тексте стоп-слов повышают точность на пару процентов. При этом лучшей получается модель с большим диапазоном n-грамм (размер полученных векторов существенно больше в этом случае)

**Вывод:** при использовании мешка n-грамм имеет смысл полная предобработка текста по данному выше списку. Пусть точность немного меньше максимальной полученной тут, но количество использованной памяти и время расчета меньше, чем в случае отсутствия предобработки. При этом, для векторизации будет достаточно использовать униграммы и биграммы

**Всилу долгих рассчетов, далее будем рассматривать только полностью предобработанные данные**

**CountVectorizer char analizer**

В некоторых задачах в качестве признаков могут быть использщованы, n-граммы символов. Для этого необходимо установить в `CountVectorizer()` параметр `analyzer = 'char'`, то есть анализировать символы.

- char analizer  &  delete stop_words  &  without punctuation

In [ ]:
analizer, stop_words, punct = 'char', noise, False
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc, y_train, X_test_proc, y_test, 4, 10, punct, (analizer, stop_words), True)

ngram size:  1 
ngram_range:  (4, 4) 
vectorized x_train shape:  (60300, 93149)
              precision    recall  f1-score   support

    neautral       0.58      0.68      0.63     10026
    negative       0.71      0.61      0.66      9819
    positive       0.84      0.82      0.83      9855

    accuracy                           0.70     29700
   macro avg       0.71      0.70      0.71     29700
weighted avg       0.71      0.70      0.70     29700

ngram size:  2 
ngram_range:  (4, 5) 
vectorized x_train shape:  (60300, 326036)
              precision    recall  f1-score   support

    neautral       0.59      0.69      0.63     10026
    negative       0.72      0.62      0.66      9819
    positive       0.86      0.83      0.84      9855

    accuracy                           0.71     29700
   macro avg       0.72      0.71      0.71     29700
weighted avg       0.72      0.71      0.71     29700

ngram size:  3 
ngram_range:  (4, 6) 
vectorized x_train shape:  (60300, 7872

In [ ]:
analizer, stop_words, punct = 'char', noise, False
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc, y_train, X_test_proc, y_test, 10, 13, punct, (analizer, stop_words), True)

ngram size:  1 
ngram_range:  (10, 10) 
vectorized x_train shape:  (60300, 2113377)
              precision    recall  f1-score   support

    neautral       0.63      0.64      0.63     10026
    negative       0.69      0.69      0.69      9819
    positive       0.86      0.84      0.85      9855

    accuracy                           0.72     29700
   macro avg       0.72      0.72      0.72     29700
weighted avg       0.72      0.72      0.72     29700

ngram size:  3 
ngram_range:  (10, 12) 
vectorized x_train shape:  (60300, 7779240)
              precision    recall  f1-score   support

    neautral       0.63      0.62      0.62     10026
    negative       0.68      0.70      0.69      9819
    positive       0.85      0.84      0.85      9855

    accuracy                           0.72     29700
   macro avg       0.72      0.72      0.72     29700
weighted avg       0.72      0.72      0.72     29700

ngram size:  1 
ngram_range:  (11, 11) 
vectorized x_train shape:  (60

- char analizer  &  delete stop_words  &  with punctuation

In [ ]:
analizer, stop_words, punct = 'char', noise, True
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc, y_train, X_test_proc, y_test, 4, 10, punct, (analizer, stop_words))

----------------------
BEST:
ngram_range (5, 9) 
               precision    recall  f1-score   support

    neautral       0.61      0.68      0.65     10026
    negative       0.72      0.65      0.68      9819
    positive       0.87      0.85      0.86      9855

    accuracy                           0.73     29700
   macro avg       0.74      0.73      0.73     29700
weighted avg       0.73      0.73      0.73     29700



Замечательная особенность признаков-символов - для них не нужна токенизация и лемматизация, можно использовать такой подход для языков, у которых нет готовых анализаторов.

- without preprocessing & char analizer  &  not delete stop_words  &  with punctuation

In [ ]:
analizer, stop_words, punct = 'char', None, True
get_models(get_count_vectorizer, generate_ngram_full_range, X_train, y_train, X_test, y_test,  4, 10, punct, (analizer, stop_words))

----------------------
BEST:
ngram_range (5, 9) 
               precision    recall  f1-score   support

    neautral       0.62      0.68      0.65     10026
    negative       0.72      0.66      0.69      9819
    positive       0.88      0.86      0.87      9855

    accuracy                           0.73     29700
   macro avg       0.74      0.73      0.73     29700
weighted avg       0.74      0.73      0.73     29700



- На вообще необработанных данных модель дает хорошую точность, сравнимую с обработанными, что делает ее полезной в ситуациях, где обработка данных трудоемка.

- На данных, полностью предобработанных по списку, при увеличении диапазона размеров n-грамм качество модели немного увеличивается с увеличением размерности. Но этот эффект прекращается после (10, 10) 

Лучшая модель с параметром 'char' дала результат с accuracy на пару сотых больше, чем модели с параметром 'word'. 
Видно, что размер промежутка взятых n-грамм, равен примерно паре слов, что попало в промежуток модели с параметром 'word'

**TF-IDF векторизация**

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений выдает **tf-idf** каждого слова.

Как считается tf-idf:

**TF (term frequency)** – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_{t}}{\sum_k n_{k}} $$

**IDF (inverse document frequency)** – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

Перемножаем их:
$$TFIDF(t, d, D) = TF(t,d) \times IDF(i, D)$$

**Cмысл**: если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.



```
class sklearn.feature_extraction.text.TfidfVectorizer(*, 
                                                      input='content', 
                                                      encoding='utf-8', 
                                                      decode_error='strict', 
                                                      strip_accents=None, 
                                                      lowercase=True, 
                                                      preprocessor=None, 
                                                      tokenizer=None, 
                                                      analyzer='word', 
                                                      stop_words=None, 
                                                      token_pattern='(?u)\b\w\w+\b', 
                                                      ngram_range=(1, 1), 
                                                      max_df=1.0, 
                                                      min_df=1, 
                                                      max_features=None, 
                                                      vocabulary=None, 
                                                      binary=False, 
                                                      dtype=<class 'numpy.float64'>, 
                                                      norm='l2', 
                                                      use_idf=True, 
                                                      smooth_idf=True, 
                                                      sublinear_tf=False)
```



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_tf_idf_vectorizer(X_train, y_train, X_test, y_test, vectorizer_params=(), print_res = False) :
    analyzer, stop_words, max_df, min_df, max_features, ngram_range_start, ngram_range_end = vectorizer_params
    max_features = int(X_train.shape[0] * max_features)
    ngram_range = (ngram_range_start, ngram_range_end)

    smart_vectorizer = TfidfVectorizer(analyzer = analyzer, ngram_range = ngram_range, stop_words = stop_words, max_df = max_df, min_df = min_df, max_features = max_features) 
    smart_vectorized_x_train = smart_vectorizer.fit_transform(X_train)
    smart_vectorized_x_test = smart_vectorizer.transform(X_test)

    pred, report = naive_bayes_classifier(smart_vectorized_x_train, y_train, smart_vectorized_x_test, y_test)
    if print_res :
        print('ngram size: ', ngram_range[1] - ngram_range[0] + 1, 
              '\nngram_range: ', ngram_range, 
              '\nvectorized x_train shape: ', smart_vectorized_x_train.shape, 
              '\nmax_df: ', max_df, ' min_df: ', min_df, ' max_features: ', max_features)
        print(report)

    vect_pararams = smart_vectorizer.get_params()
    params = ('TfidfVectorizer', 
              ['analyzer : ' + vect_pararams['analyzer'], 
               'ngram_range : ' + str(vect_pararams['ngram_range']),
               'max_df : ' + str(vect_pararams['max_df']),
               'min_df : ' + str(vect_pararams['min_df']), 
               'max_features : ' + str(vect_pararams['max_features'])])
    
    return pred, report, params

tf-idf (диапазон для n (ngram_range) подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)

In [ ]:
for max_df in np.arange(0.0, 1.1, 0.2) :
    for min_df in np.arange(0.0, 1.1, 0.2) :
        for max_features in np.arange(0.0, 1.1, 0.2) :
            try :
                get_models(get_tf_idf_vectorizer, generate_ngram_full_range, X_train_proc, y_train, X_test_proc, y_test, 1, 4, False, ('word', None, max_df, min_df, max_features), True)
                get_models(get_tf_idf_vectorizer, generate_ngram_full_range, X_train_proc, y_train, X_test_proc, y_test, 1, 4, True, ('word', None, max_df, min_df, max_features), True)
            except ValueError :
                continue

Выходные данные были обрезаны до нескольких последних строк (5000).
max_df:  0.4  min_df:  0.0  max_features:  24120
              precision    recall  f1-score   support

    neautral       0.60      0.64      0.62     10026
    negative       0.67      0.65      0.66      9819
    positive       0.84      0.81      0.83      9855

    accuracy                           0.70     29700
   macro avg       0.71      0.70      0.70     29700
weighted avg       0.70      0.70      0.70     29700

ngram size:  1 
ngram_range:  (3, 3) 
vectorized x_train shape:  (60300, 24120) 
max_df:  0.4  min_df:  0.0  max_features:  24120
              precision    recall  f1-score   support

    neautral       0.57      0.46      0.51     10026
    negative       0.52      0.70      0.60      9819
    positive       0.77      0.66      0.71      9855

    accuracy                           0.61     29700
   macro avg       0.62      0.61      0.61     29700
weighted avg       0.62      0.61      0.61   

При построении словаря игнорируем термины, частота которых в документе строго превышает заданный порог (стоп-слова для конкретного корпуса). Поскольку стоп-слова, как правило, имеют высокую частоту, возможно, имеет смысл использовать max_df. Например, значение 0.95, чтобы удалить верхние 5% самых частотных слов, тогда мы предполагаем, что верхние 5% - это все стоп-слова, что может быть не так. Это зависит от текстовых данных. (Мы можем удалять выбранные стоп слова из текста, или же использовать параметр max_df)

При построении словаря игнорируем термины, частота которых в документе строго ниже заданного порогового значения. min_df, чтобы удалить редкие слова. Если они встречаются только один или два раза, они не добавят большой ценности и. Также, их, как правило, много, поэтому игнорирование их с помощью min_df может значительно уменьшить потребление памяти и размер данных.

In [ ]:
print(len(report_list))
for i in range(len(report_list)) :
  print(i, ' ', report_list[i])

108
0   {'Vectorizer': 'CountVectorizer', 'Vectorizer_parameters': ['analyzer : word', 'ngram_range : (1, 2)'], 'Use punctuation': False, 'Precision': 0.7133214517397306, 'Recall': 0.7136608739057166, 'F1-score': 0.7129385894792821, 'Accuracy': 0.7132323232323232}
1   {'Vectorizer': 'CountVectorizer', 'Vectorizer_parameters': ['analyzer : word', 'ngram_range : (1, 2)'], 'Use punctuation': True, 'Precision': 0.7142981521278847, 'Recall': 0.7145653009224274, 'F1-score': 0.7138812126629875, 'Accuracy': 0.7141414141414142}
2   {'Vectorizer': 'CountVectorizer', 'Vectorizer_parameters': ['analyzer : word', 'ngram_range : (1, 6)'], 'Use punctuation': False, 'Precision': 0.7350509084835265, 'Recall': 0.7331575235978686, 'F1-score': 0.734043157077238, 'Accuracy': 0.7326599326599327}
3   {'Vectorizer': 'CountVectorizer', 'Vectorizer_parameters': ['analyzer : word', 'ngram_range : (1, 3)'], 'Use punctuation': True, 'Precision': 0.7357376871177873, 'Recall': 0.7329841536570513, 'F1-score': 0.73412

В выводе сохранился запуск для TfidfVectorizer, где stop_words не равен None (значения до индекса 58) Далее идут записи для stop_words = None

In [ ]:
def generate_df() :
    df = pd.DataFrame(report_list)
    df['score'] = (df['Precision'] + df['Recall'] + df['F1-score'] + df['Accuracy']) / 4
    return df.sort_values('score', ascending=False)

In [ ]:
result_df = generate_df()
result_df.head(5)

,Vectorizer,Vectorizer_parameters,Use punctuation,Precision,Recall,F1-score,Accuracy,score
3,CountVectorizer,"[analyzer : word, ngram_range : (1, 3)]",True,0.735738,0.732984,0.734124,0.732593,0.733860
2,CountVectorizer,"[analyzer : word, ngram_range : (1, 6)]",False,0.735051,0.733158,0.734043,0.732660,0.733728
82,TfidfVectorizer,"[analyzer : word, ngram_range : (1, 2), max_df...",False,0.738877,0.731304,0.733201,0.731212,0.733649
92,TfidfVectorizer,"[analyzer : word, ngram_range : (1, 2), max_df...",False,0.738877,0.731304,0.733201,0.731212,0.733649
93,TfidfVectorizer,"[analyzer : word, ngram_range : (1, 2), max_df...",True,0.738877,0.731304,0.733201,0.731212,0.733649


Параметры TfidfVectorizer у лучших моделей:

82) 'max_df : 0.6000000000000001', 'min_df : 0.0',

92) 'max_df : 0.8', 'min_df : 0.0',

93) 'max_df : 0.8', 'min_df : 0.0'

По ним можно сделать вывод, что в тексте мало специфических слов, которые ухудшают работу модели. 
Так, min_df : 0.0 - мы вообще не отрезаем редкие слова, а у лучшей из этих моделей параметр max_df : 0.6, то есть частые слова мы из текста убираем жестко - 40% слов отрезается в лучшей модели. (возможно, в этот список попали и стоп-слова)


Наилучший результат показал CountVectorizer с analyzer : word.
(пропустила сохранение stop_words для векторайзеров в dataframe, поэтому тут отследить сложно, где именно полностью предобработанные данные, а где без удаления стоп слов - это можно найти в логах, BEST случаи сохранялись в таблицу)
Параметры вызова лучших моделей:

```
analizer, stop_words, punct = 'word', None, False
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc, y_train, X_test_proc, y_test, 1, 7, False, ('word', None))

analizer, stop_words, punct = 'word', None, True
get_models(get_count_vectorizer, generate_ngram_full_range, X_train_proc_punct, y_train, X_test_proc_punct, y_test, 1, 7, True, ('word', None))
```

TfidfVectorizer оказался не сильно хуже, поэтому в рамках нашей задачи нет принципиальной разницы, какой векторайзер использовать для обработки.

Также, можем заметить. что наиболее точные модели не удаляют из текста стоп слова. И используют различные значения ngram_range в зависимости от наличия пунктуации в тексте. Учитывая, что для ngram_range = (1, 6) строится очень много вспомогательных векторов, использование данной модели нерационально, так как ее точность практически равна модели с ngram_range = (1, 3) и убранной пунктуацией. Скорее всего такой результат получен из-за того, что знаки препинания в тексте представляют собой отдельные токены, которые считаются моделью, можно сделать вывод, что пунктуация в нашем тексте не важна. Ее присутствие лишь усложняет необходимую модель.

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

Так как возвращает список всех найденных -непересекающихся- совпадений

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
def get_each_two_first_letters(string) :
  return re.findall(r'\b[а-яёА-Яa-zA-Z][а-яёА-Яa-zA-Z]', string)

In [ ]:
print(get_each_two_first_letters('А мы хотим получить по 2 первые буквы из каждого слова тут.'))

['мы', 'хо', 'по', 'по', 'пе', 'бу', 'из', 'ка', 'сл', 'ту']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
def get_no_more_than_three_sentences(text) :
    return re.split('\.', text, maxsplit = 2)

In [ ]:
print(get_no_more_than_three_sentences('Строка. в. которой. в. качестве. разделителя. использована. точка.'))

['Строка', ' в', ' которой. в. качестве. разделителя. использована. точка.']


In [ ]:
print(get_no_more_than_three_sentences('Строка, в которой в качестве разделителя. использована только одна точка'))

['Строка, в которой в качестве разделителя', ' использована только одна точка']


### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
def from_digits_to_dig(string) :
    return re.sub('\d', 'DIG', string)

In [ ]:
print(from_digits_to_dig('Ст1ро7ка 4с4 числ78ам9и   6'))

СтDIGроDIGка DIGсDIG числDIGDIGамDIGи   DIG


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
#'^(https?://)?       ([\da-z\.-]+) \.       ([a-z\.]{2,6})    ([\/\w \.-]*)  */?$'

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
def get_words_longer_than_three(string) :
    prog = re.compile('[А-Яа-яё\-]{4,}')
    return prog.findall(string)

In [ ]:
print(get_words_longer_than_three('Слова? Да, больше, ещё больше слов! Что-то ещё.'))

['Слова', 'больше', 'больше', 'слов', 'Что-то']


**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
def find_domens(string) :
    prog = re.compile('@[\da-z_\.-]+\.[a-z\.]{2,6}')
    return prog.findall(string)

In [ ]:
print(find_domens('abc.test@gma8il.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz'))

['@gma8il.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']
